# **Crypto On-chain Data Analysis**
This notebook analyzes historical tokenomics for major cryptocurrencies (BTC, ETH, BNB, SOL) using data from CoinGecko API.

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pycoingecko import CoinGeckoAPI

# Initialize CoinGecko API client
cg = CoinGeckoAPI()

# Set default template for plotly
import plotly.io as pio
pio.templates.default = "plotly_dark"


In [2]:
# Define the cryptocurrencies we want to analyze
crypto = pd.DataFrame({
    'symbol': ['BTC', 'ETH', 'BNB', 'SOL', 'XRP', 'AVAX'],
    'name': ['Bitcoin', 'Ethereum', 'Binance Coin', 'Solana', 'XRP', 'Avalanche'],
    'coingecko_id': ['bitcoin', 'ethereum', 'binancecoin', 'solana', 'ripple', 'avalanche-2']
})

print("Cryptocurrencies to analyze:")
print(crypto)

Cryptocurrencies to analyze:
  symbol          name coingecko_id
0    BTC       Bitcoin      bitcoin
1    ETH      Ethereum     ethereum
2    BNB  Binance Coin  binancecoin
3    SOL        Solana       solana
4    XRP           XRP       ripple
5   AVAX     Avalanche  avalanche-2


In [ ]:
# Fetching Tokenomics Data
tokenomics_data = []

for _, row in crypto.iterrows():
    try:
        print(f"Fetching data for {row['name']}...")
        # Fetch detailed coin data from CoinGecko
        coin_data = cg.get_coin_by_id(row['coingecko_id'])
        data = {
            'symbol': row['symbol'],
            'current_price': coin_data['market_data']['current_price']['usd'],
            'market_cap': coin_data['market_data']['market_cap']['usd'],
            'total_volume': coin_data['market_data']['total_volume']['usd'],
            'circulating_supply': coin_data['market_data']['circulating_supply'],
            'total_supply': coin_data['market_data']['total_supply'],
            'max_supply': coin_data['market_data']['max_supply'],
            'fdv': coin_data['market_data'].get('fully_diluted_valuation', {}).get('usd'),
        }
        tokenomics_data.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {row['symbol']}: {e}")

# Convert to DataFrame
tokenomics_df = pd.DataFrame(tokenomics_data)

Fetching data for Bitcoin...
Fetching data for Ethereum...
Fetching data for Binance Coin...
Fetching data for Solana...
Fetching data for XRP...
Fetching data for Avalanche...


In [4]:
# Fill NaN values in max_supply with total_supply values
tokenomics_df['max_supply'] = tokenomics_df['max_supply'].fillna(tokenomics_df['total_supply'])

# Display the updated dataframe to verify changes
print("\nUpdated max supply values:")
print(tokenomics_df[['symbol', 'total_supply', 'max_supply']])


Updated max supply values:
  symbol  total_supply    max_supply
0    BTC  1.985835e+07  2.100000e+07
1    ETH  1.207329e+08  1.207329e+08
2    BNB  1.458876e+08  2.000000e+08
3    SOL  5.997663e+08  5.997663e+08
4    XRP  9.998621e+10  1.000000e+11
5   AVAX  4.536344e+08  7.200000e+08


In [5]:
# 2. Supply Metrics Comparison (Grouped Bar Chart)
supply_data = pd.melt(
    tokenomics_df,
    id_vars=['symbol'],
    value_vars=['circulating_supply', 'total_supply', 'max_supply'],
    var_name='supply_type',
    value_name='supply',
)
fig2 = px.bar(
    supply_data,
    x='symbol',
    y='supply',
    color='supply_type',
    title='<b>Supply Metrics by Cryptocurrency</b>',
    barmode='group',
    labels={'supply': 'Supply Amount', 'symbol': 'Cryptocurrency'}
)

# Customize the layout
fig2.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
    yaxis=dict(
        type='log',
        tickvals=[1e6, 1e7, 1e8, 1e9, 1e10, 1e11],  # 1M, 10M, 100M, 1B, 10B, 100B
        ticktext=['1M', '10M', '100M', '1B', '10B', '100B'],
        showgrid=True
    ),
    
)

fig2.show()

In [6]:
# 3. Market Cap vs Volume (Scatter Plot)
fig3 = px.scatter(
    tokenomics_df,
    x='market_cap',
    y='total_volume',
    text='symbol',
    title='<b>Market Cap vs Trading Volume</b>',
    labels={
        'market_cap': 'Market Cap (USD)',
        'total_volume': 'Trading Volume (USD)'
    },
    size='market_cap',
    size_max=150,
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
    
)

# Customize the layout
fig3.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig3.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [7]:
# 5. Market Dominance (Pie Chart)
fig5 = px.pie(
    tokenomics_df,
    values='market_cap',
    names='symbol',
    title='<b>Market Cap Distribution</b>',
    subtitle='From Selected Cryptocurrencies',
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
)

# Customize the layout
fig5.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig5.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [15]:
# Update the format_number function to include trillions
def format_number(value):
    """Format large numbers to abbreviated string with T, B, M"""
    if value >= 1e12:
        return f'{value/1e12:.1f}T'
    elif value >= 1e9:
        return f'{value/1e9:.1f}B'
    elif value >= 1e6:
        return f'{value/1e6:.1f}M'
    else:
        return f'{value:.0f}'

# Format the values in the DataFrame
tokenomics_df['fdv_formatted'] = tokenomics_df['fdv'].apply(format_number)
tokenomics_df['volume_formatted'] = tokenomics_df['total_volume'].apply(format_number)
tokenomics_df['market_cap_formatted'] = tokenomics_df['market_cap'].apply(format_number)
tokenomics_df['circulating_supply_formatted'] = tokenomics_df['circulating_supply'].apply(format_number)
tokenomics_df['total_supply_formatted'] = tokenomics_df['total_supply'].apply(format_number)

In [16]:
# Create 3D scatter plot
fig6 = px.scatter(
    tokenomics_df,
    x='total_supply',
    y='circulating_supply',
    size='market_cap',
    color='symbol',
    size_max=60,
    title='<b>Crypto Tokenomics</b>',
    subtitle='Bubble size defined by Market Cap.',
    labels={
        'fdv': 'FDV',
        'circulating_supply': 'Circulating Supply',
        'total_supply': 'Total Supply',
    },
    color_discrete_map={
        'BTC': '#F7931A',
        'ETH': '#636BFF',
        'BNB': '#F3BA2F',
        'SOL': '#CF62E7',
        'XRP': '#4B8BBE',
        'AVAX': '#E84142'
    },
    custom_data=['symbol','market_cap_formatted', 'fdv_formatted', 'circulating_supply_formatted', 'total_supply_formatted']  # Add formatted values    
)

# Update hover template
fig6.update_traces(
    hovertemplate="<br>".join([
        "<b>%{customdata[0]}</b>",  # Crypto Symbol
        "<b>$%{customdata[1]}</b> - Market Cap",
        "<b>$%{customdata[2]}</b> - FDV",
        "<b>%{customdata[3]}</b> - Circulating Supply",
        "<b>%{customdata[4]}</b> - Total Supply",
        "<extra></extra>"
    ]),
    hoverlabel=dict(
        font=dict(
            family="Inter",
            size=12,
        ),
        bordercolor="rgba(17, 17, 17, 0.9)",  # Same as background to remove border
        namelength=-1  # Show full text
    ),
)

# Customize the layout
fig6.update_layout(
    template='plotly_dark',
    font_family="Inter",
    
    height=435,  # Adjust height
    width=705,  # Adjust width
    margin=dict(
        l=70,
        r=70,
        t=70,
        b=70 
    ),
    legend=dict(
        title=dict(text="Cryptocurrency"),
        font=dict(
            family="IBM Plex Sans",
            size=11,
            color="white"
        ),
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.15,
    ),
    hovermode="x",
)

# Show the plot
fig6.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig6.write_html("plots/html/crypto_tokenomics.html")

In [19]:
# Create 3D scatter plot
fig6m = px.scatter(
    tokenomics_df,
    x='total_supply',
    y='circulating_supply',
    size='market_cap',
    color='symbol',
    size_max=60,
    title='Crypto Tokenomics',
    labels={
        'fdv': 'FDV',
        'circulating_supply': 'Circulating Supply',
        'total_supply': 'Total Supply',
    },
    color_discrete_map={
        'BTC': '#F7931A',
        'ETH': '#636BFF',
        'BNB': '#F3BA2F',
        'SOL': '#CF62E7',
        'XRP': '#4B8BBE',
        'AVAX': '#E84142'
    },
    custom_data=['symbol','market_cap_formatted', 'fdv_formatted', 'circulating_supply_formatted', 'total_supply_formatted']  # Add formatted values    
)

# Update hover template
fig6m.update_traces(
    hovertemplate="<br>".join([
        "<b>%{customdata[0]}</b>",  # Crypto Symbol
        "<b>$%{customdata[1]}</b> - Market Cap",
        "<b>$%{customdata[2]}</b> - FDV",
        "<b>%{customdata[3]}</b> - Circulating Supply",
        "<b>%{customdata[4]}</b> - Total Supply",
        "<extra></extra>"
    ]),
    hoverlabel=dict(
        font=dict(
            family="Inter",
            size=12,
        ),
        bordercolor="rgba(17, 17, 17, 0.9)",  # Same as background to remove border
        namelength=-1  # Show full text
    ),
)

# Customize the layout
fig6m.update_layout(
    template='plotly_dark',
    font_family="IBM Plex Sans",
    showlegend=False,
    height=393,  # Adjust height
    width=333,  # Adjust width
    margin=dict(
        l=10,
        r=10,
        t=40,
        b=10 
    ),
    title=dict(
        text='Crypto Tokenomics',
        font=dict(size=12, color='white'),
        xanchor='left'
    ),
    legend=dict(
        title=dict(text="Cryptocurrency"),
        font=dict(
            family="Inter",
            size=11,
            color="white"
        ),
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.15,
    ),
    hovermode="x",
)

# Show the plot
fig6m.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig6m.write_html("plots/html/crypto_tokenomics_mobile.html")